In [ ]:
%load_ext autoreload
%autoreload 1

In [1]:
import os
import sys

sys.path.insert(
    0,
    os.path.join(
        os.environ["PROJ_fe_graph"],
        "data",
        ),
    )

import numpy as np
from ase.visualize import view

from ase import io
import json

import itertools

from proj_data_fe_graph import (
    most_stable_crystal_structure_dict,
    exp_latt_const_dict,
    )

# Script Inputs

In [2]:
latt_const_array_len = 5

element_list = [
    "Ni",
#     "Co",
#     "Ru",
#     "Rh",
#     "Mo",
#     "W",
    ]

# Methods

In [3]:
def create_latt_const_range(latt_const_0, latt_const_range_perc=0.05, num=100):
    d_lc = latt_const_range_perc * latt_const_0
    latt_const_range = np.linspace(
        latt_const_0 - d_lc,
        latt_const_0 + d_lc,
        num=num,
        endpoint=True,
        )

    return(latt_const_range)

In [4]:
master_job_list = []
for elem_i in element_list:
    latt_const_dict_i = exp_latt_const_dict[elem_i]
    crys_struct_i = most_stable_crystal_structure_dict[elem_i]
    exp_lattices_i = latt_const_dict_i[crys_struct_i]

    #| - Creating permutations of all lattice constants
    lattice_constant_array_dict = {}
    for key, value in exp_lattices_i.items():
        latt_const_range_j = create_latt_const_range(
            value,
            num=latt_const_array_len,
            )
        latt_const_range_j = np.round(
            latt_const_range_j,
            decimals=5
            )
        lattice_constant_array_dict[key] = latt_const_range_j

    latt_const_permut_i = list(
        itertools.product(
            *list(
                lattice_constant_array_dict.values()
                )
            )
        )
    #__|


    for lattice_constants_k in latt_const_permut_i:
        
        for spinpol in [True, False]:
            job_dict_i = dict(
                zip(
                    list(exp_lattices_i.keys()),
                    list(lattice_constants_k),
                    )
                )

            job_dict_i["support_metal"] = elem_i
            job_dict_i["crystal_structure"] = crys_struct_i
            job_dict_i["spinpol"] = spinpol

            path_1_i = os.path.join(
                job_dict_i.get("support_metal", "nan"),
                job_dict_i.get("crystal_structure", "nan"),
                str(job_dict_i.get("spinpol", "nan")),
                )

            path_i = os.path.join(
                "data",
                path_1_i,
                str(job_dict_i.get("a", "nan")) + "-" + str(job_dict_i.get("c", "nan")),
                )
            
            master_job_list.append(
                {
                    "properties": job_dict_i,
                    "path": path_i,
                    }
                )

# Instantiate the Jobs class instance

In [6]:
from dft_job_automat.job_setup import DFT_Jobs_Setup

Jobs_S = DFT_Jobs_Setup(
    tree_level=None,
    level_entries=None,
    indiv_dir_lst=None,
    indiv_job_lst=None,

    skip_dirs_lst=None,
    root_dir=".",
    working_dir=".",
    folders_exist=None,
    parse_all_revisions=True,
    )

Jobs_S.create_Jobs_from_dicts_and_paths(master_job_list)
Jobs_S.create_dir_struct()
Jobs_S.write_job_params_json_file()

# Running Job Setup

from methods import dir_setup
print(25 * "*")
atoms_list = []
for Job_i in Jobs_S.Job_list:
    path_i = Job_i.full_path
    job_i_params = Job_i.job_params_dict

    atoms_i = dir_setup(
        path_i,
        job_i_params,
        )

    atoms_list.append(atoms_i)

# Job Manager Instance

In [8]:
from dft_job_automat.job_manager import DFT_Jobs_Manager

Jobs_M = DFT_Jobs_Manager(
#     tree_level=None,
#     level_entries=None,
#     skip_dirs_lst=None,
#     indiv_dir_lst=None,  # <-----------------------------------------------
#     indiv_job_lst=None,
#     root_dir=".",
#     working_dir=".",
#     update_job_state=False,
    load_dataframe=False,
#     dataframe_dir=None,
#     job_type_class=None,
#     methods_to_run=None,
#     folders_exist=None,
#     parse_all_revisions=True,
    )

Jobs_M.create_Jobs_from_dicts_and_paths(master_job_list)

for Job_i in Jobs_M.Job_list:
    Jobs_M.submit_job(**{
        "nodes": "1",  # --nodes
        "cpus": "4",  # --ntasks-per-node
        "memory": "2000",  # --mem-per-cpu
        "wall_time": "30",  # --time (720min -> 12hrs)

#         "queue": "owners,iric,normal",  # -p flag
#         "job_name": "Default",  # --job-name
#         "priority": "normal",  # --qos
        # "email": "flores12@stanford.edu",  # --mail-user
#         "email": self.username + "@stanford.edu",  # --mail-user
#         "email_mess": "FAIL",  # --mail-type    
        })
